In [ ]:
import os
import requests
import json
from bs4 import BeautifulSoup as Soup
import pandas as pd

In [ ]:
DIR = 'XMLs'
def ensure_dir(d):
   d = os.path.join(d, '')
   if not os.path.exists(d):
       os.makedirs(d)
ensure_dir(DIR)

In [ ]:
query = "tourism+AND+innovation"
apiKey = ""

In [ ]:
data = {'Title': [],
        'Authors': [],
        'PublicationName': [],
        'Type': [],
        'Subtype': [],
        'Abstract': [],
        'Volume': [],
        'Issue': [],
        'Page Range': [],
        'Date': [],
        'EID': [],
        'DOI': [],
        'Keywords' : [],
        'URL' : [],
        'OpenAccess': [],
        'References': [],
        'CitedBy': [],
        'Author 1': [],
        'Author 2': [],
        'Author 3': [],
        'Author 4': [],
        'Author 5': [],
        'Author 6': [],
        'Author 7': [],
        'Author 8': [],
        'Author 9': [],
        'Author 10': [],
        'AuthorAUID': [],
        'Affiliation Name': [],
        'Affilication City': [],
        'Affiliation Country': [],
        'Affiliation ID': [],
        'SubjectAreas': [],
        'Scopus Cited By': [],
        'Scopus Identifier': [],
        'Funding Acronym': [],
        'Funding Agency ID': [],
        'Funding Agency': []
        }

In [ ]:
cursor = "*"
count = 0

while cursor:
    print(count)
    response = requests.get("https://api.elsevier.com/content/search/scopus?query=TITLE-ABS-KEY({})&apiKey={}&cursor={}&count=25"
                            .format(query, apiKey, cursor))
    cont = json.loads(response.content)
    if cont["search-results"]["cursor"]["@next"] != cursor:
        cursor = cont["search-results"]["cursor"]["@next"].replace('+','%2B').replace('/', '%2F')
    else:
        cursor = None
    if "entry" not in cont["search-results"]:
        break
    for i in cont["search-results"]["entry"]:
        url = i['prism:url']
        data['URL'].append(str(url))
        aid = url.split('/')[-1]
        filename = os.path.join(DIR, aid + '.xml')
        if os.path.isfile(filename):
            with open(filename) as f:
                article = f.read()
        else:
            article = requests.get("{}?apiKey={}".format(url, apiKey)).text
            with open(filename, 'w') as f:
                f.write(article)
        soup = Soup(article, features="lxml")
        
        data['Title'].append('"""' + soup.find('dc:title').get_text().replace('\n', ' ').replace('\t', ' ') + '"""'
                             if soup.find('dc:title') else '')
        authors = soup.find('authors')
        data['Authors'].append('; '.join([item.get_text() for item in authors.find_all('ce:indexed-name') 
                                          if item.parent.name == 'author']) if authors else '')
        data['PublicationName'].append(soup.find('prism:publicationname').get_text() if 
                                       soup.find('prism:publicationname') else '')
        data['Type'].append(soup.find('prism:aggregationtype').get_text() if
                            soup.find('prism:aggregationtype') else '')
        data['Subtype'].append(soup.find('subtypedescription').get_text() if soup.find('subtypedescription') else '')
        data['Abstract'].append('"""' + soup.find('ce:para').get_text().replace('\n', ' ').replace('\t', ' ') + '"""'
                                if soup.find('ce:para') and soup.find('ce:para').parent.name == 'abstract' else '')
        data['Volume'].append(soup.find('prism:volume').get_text() if soup.find('prism:volume') else '')
        data['Issue'].append(soup.find('prism:issueidentifier').get_text() 
                             if soup.find('prism:issueidentifier') else '')
        data['Page Range'].append(soup.find('prism:pagerange').get_text() if soup.find('prism:pagerange') else '')
        data['Date'].append(soup.find('prism:coverdate').get_text() if soup.find('prism:coverdate') else '')
        data['EID'].append(soup.find('eid').get_text() if soup.find('eid') else '')
        data['DOI'].append(soup.find('prism:doi').get_text() if soup.find('prism:doi') else '')
        data['Keywords'].append('; '.join([label.get_text() for label in soup.find_all('author-keyword') 
                                             if label.parent.name == 'author-keywords']))
        data['OpenAccess'].append(soup.find('openaccess').get_text() if soup.find('openaccess') else '')
        data['References'].append('; '.join([label.get_text().replace('\t', ' ') for label in soup.find_all('ref-fulltext')]) 
                                  if soup.find_all('ref-fulltext') else '')
        data['CitedBy'].append(soup.find('citedby-count').get_text() if soup.find('citedby-count') else '')
        if soup.find('authors'):
            authors = soup.find('authors').find_all('author')
        else:
            authors = None
        for i in range(10):
            if authors and i < len(authors):
                author = authors[i]
                data['Author {}'.format(i+1)].append('; '.join([author.find('ce:indexed-name').get_text(),
                                                                author.find('ce:given-name').get_text(),
                                                                author.find('ce:surname').get_text(),
                                                                author.find('author-url').get_text(),
                                                                author['auid'],
                                                                author.find('affiliation')['id'] 
                                                                if author.find('affiliation') else '']).replace('\t', ' '))
            else:
                data['Author {}'.format(i+1)].append('')                                   
        data['AuthorAUID'].append('; '.join([auth['auid'] for auth in soup.find_all('author') 
                                                 if auth.parent.name == 'authors']) if soup.find_all('author') else '')
        data['Affiliation Name'].append('; '.join([name.get_text() for name in soup.find_all('affilname')]) 
                                                  if soup.find_all('affilname') else '')
        data['Affilication City'].append('; '.join([name.get_text() for name in soup.find_all('affiliation-city')]) 
                                                  if soup.find_all('affiliation-city') else '')
        data['Affiliation Country'].append('; '.join([name.get_text() for name in soup.find_all('affiliation-country')]) 
                                                  if soup.find_all('affiliation-country') else '')
        data['Affiliation ID'].append('; '.join([i.get_text() for i in soup.find_all('afid')]) 
                                      if soup.find_all('afid') else '')
        data['SubjectAreas'].append('; '.join([label.get_text().replace('\t', ' ') for label in soup.find_all('subject-area') 
                                                 if label.parent.name == 'subject-areas']) 
                                      if soup.find_all('subject-area') else '')
        data['Scopus Cited By'].append(soup.find(rel='scopus-citedby')['href'] 
                                       if soup.find(rel='scopus-citedby') else '')
        data['Scopus Identifier'].append(soup.find('dc:identifier').get_text() if soup.find('dc:identifier') else '')
        data['Funding Acronym'].append(soup.find('fund-acr').get_text() if soup.find('fund-acr') else '')
        data['Funding Agency ID'].append(soup.find('fund-no').get_text() if soup.find('fund-no') else '')
        data['Funding Agency'].append(soup.find('fund-sponsor').get_text() if soup.find('fund-sponsor') else '')
        count += 1
    df = pd.DataFrame.from_dict(data)
    df.to_csv(os.path.join(os.getcwd(), '../Innovation/Innovation-Scopus-backup.csv'), index=False)
    df.to_csv(os.path.join(os.getcwd(), '../Innovation/Innovation-Scopus.csv'), index=False)